In [46]:
import math
import h5py
import numpy as np 
import netCDF4 as nc
from netCDF4 import Dataset
from mpl_toolkits.basemap import Basemap
import xarray as xr
import matplotlib.pyplot as plt
import time
import pandas as pd
import datetime

In [85]:
#---Kristian

# calculate area-weighted mean for each region

# average winter months precipitation for each region

# change units to mm/day (length/time) instead of m/s


#---Anthony
# repeat process for all simulations and years
# save as netcdf

# sanity plots

In [168]:
dataframes2 = pd.DataFrame(columns=['year', 'simulation', 'region', 'PRECT'])

def process_region(data, region, lat_slice, lon_slice):
    # Weight and average for the specified region
    ds_region = data.sel(lat=lat_slice, lon=lon_slice)
    weights_region = np.cos(np.deg2rad(ds_region['lat']))
    weights_region.name = 'weights'
    ds_weighted_region = ds_region['PRECT'].weighted(weights_region)
    mean_sst_region = ds_weighted_region.mean(dim=['lat', 'lon'])
    ds_region = mean_sst_region

    # Filter months of interest (Nov, Dec, Jan, Feb, Mar)
    month_list = [True if month in [11, 12, 1, 2, 3] else False for month in data['time.month']]
    ds_region = ds_region.loc[{'time': month_list}] 

    # Create a new variable for grouping Nov, Dec and Jan, Feb, March from the following year
    ds_region['year_group'] = (
        ds_region['time.year'] + (ds_region['time.month'] >= 11) - 1
    ).astype(str)

    # Calculate mean PRECT for each year
    ds_region = ds_region.groupby('year_group').mean(dim='time')

    # Convert m/s to mm/day (1 m/s = 86400000 mm/day)
    ds_region = ds_region * 86400000

    # Append to the DataFrame
    df = pd.DataFrame({
        'year': ds_region.coords['year_group'].values,
        'simulation': data.attrs['case'][-8:],
        'region': region,
        'PRECT': ds_region.values
    })

    return df

def calc_prect(PRECT_data_fn):

    global dataframes2
    
    ds = xr.open_dataset(PRECT_data_fn, decode_times=False)

    reference_date = pd.to_datetime("1850-01-01")
    start_date = reference_date + datetime.timedelta(days = float(ds.time[0]))

    ds['time'] = pd.date_range(start=start_date, periods=ds.sizes['time'], freq='MS')

    south_df = process_region(ds, 'South', slice(32.0, 34.9), slice(239.4, 245.6))
    central_df = process_region(ds, 'Central', slice(34.9, 38.6), slice(236.9, 243.1))
    north_df = process_region(ds, 'North', slice(38.8, 42.2), slice(235.6, 240.6))

    # Append the DataFrames to your main list
    dataframes2 = pd.concat([dataframes2, north_df, central_df, south_df])
    return dataframes2

In [169]:
# example usage
fn = '/Users/kristianolsson/Desktop/DS6013_CC/b.e21.BHISTcmip6.f09_g17.LE2-1001.001.cam.h0.PRECT.194001-194912.nc'
output = calc_prect(fn)
output

,year,simulation,region,PRECT
0,1939,1001.001,North,6.953454
1,1940,1001.001,North,3.153461
2,1941,1001.001,North,7.083086
3,1942,1001.001,North,4.020622
4,1943,1001.001,North,2.804172
5,1944,1001.001,North,6.342207
6,1945,1001.001,North,4.810180
7,1946,1001.001,North,4.847743
8,1947,1001.001,North,3.233327
9,1948,1001.001,North,4.957548


In [170]:
# convert the DataFrame to an xarray.Dataset

output_dataset = xr.Dataset(
    {'PRECT': (['index'], output['PRECT'])},
    coords={
        'year': (['index'], output['year']),
        'simulation': (['index'], output['simulation']),
        'region': (['index'], output['region'])
    }
)

output_dataset['year'] = output['year'].unique()
output_dataset['simulation'] = output['simulation'].unique()
output_dataset['region'] = output['region'].unique()


In [171]:
output_dataset

<xarray.Dataset>
Dimensions:     (index: 33, year: 11, simulation: 1, region: 3)
Coordinates:
  * year        (year) object '1939' '1940' '1941' ... '1947' '1948' '1949'
  * simulation  (simulation) object '1001.001'
  * region      (region) object 'North' 'Central' 'South'
Dimensions without coordinates: index
Data variables:
    PRECT       (index) float64 6.953 3.153 7.083 4.021 ... 0.4809 1.516 1.221

In [172]:
# save as netcdf
output_dataset.to_netcdf('/Users/kristianolsson/Desktop/DS6013_CC/prect_data.nc')